In [1]:
import numpy as np
import pandas as pd
import itertools
import random
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm
from pathlib import Path
import warnings
import gc
import torch
import os
from copy import deepcopy
from torch import nn
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import Dataset, DataLoader
from pathlib import Path
from dataclasses import dataclass, field
import wandb
from dataclasses import asdict
from typing import Any
from scipy.stats import kendalltau
import json


GPU = "cuda:0"

In [2]:
rootdir = Path().resolve().parent.parent
inputdir = rootdir / "data" / "predict-ai-model-runtime"
embeddir = Path().resolve() / "out" / "ranknet" / "embeddings"
embeddir2 = rootdir / "data" / "google-fast-vs-slowlayout9-embedding"
workdir = Path().resolve() / "out" / "finetuning-listmle"
workdir.mkdir(exist_ok=True, parents=True)

In [3]:
dataset_dict = {}

for ds in ["train", "valid", "test"]:
    records = []
    for arch, perm in itertools.product(["nlp", "xla"], ["default", "random"]):
        datadir = inputdir / f"npz_all/npz/layout/{arch}/{perm}/{ds}"
        for filepath in sorted(datadir.glob("*.npz")):
            filename = str(filepath).split("/")[-1].replace(".npz", "")
            records.append(
                {
                    "arch": arch,
                    "perm": perm,
                    "filename": filename,
                    "filepath": filepath,
                    "embed_filepath": embeddir / arch / perm / ds / f"{filename}.npz",
                    "embed_filepath2": embeddir2 / arch / perm / ds / f"{filename}.npz",
                }
            )
    dataset_dict[ds] = pd.DataFrame(records)

In [4]:
for ds in dataset_dict:
    indexes = []
    for i, row in dataset_dict[ds].iterrows():
        try:
            np.load(row["filepath"])
            np.load(row["embed_filepath"])
            np.load(row["embed_filepath2"])
            indexes.append(i)
        except FileNotFoundError as e:
            print(row["embed_filepath2"])

    dataset_dict[ds] = dataset_dict[ds].iloc[indexes].reset_index(drop=True)

In [5]:
sheet_dict = pd.read_excel(
    "/home/yamaguchi/kaggle/data/clustering-1112.xlsx", sheet_name=None
)
train_valid_dataset = pd.concat(
    [dataset_dict["train"], dataset_dict["valid"]], axis=0, ignore_index=True
)
group_dict = {}
for group_name in sheet_dict:
    if not group_name.startswith("group"):
        continue
    dfgroup = sheet_dict[group_name].query("use == 1")[["arch", "perm", "filename"]]
    dataset = dfgroup.merge(train_valid_dataset, on=["arch", "perm", "filename"])
    sheet_dict[group_name]
    print(group_name, dataset.shape, dfgroup.shape)
    if dataset.shape[0] > 0:
        group_dict[group_name] = dataset

group17 (18, 6) (18, 3)
group16 (30, 6) (30, 3)
group15 (50, 6) (50, 3)
group10 (22, 6) (22, 3)
group9 (19, 6) (19, 3)
group8 (21, 6) (21, 3)
group7 (10, 6) (10, 3)
group6 (10, 6) (10, 3)
group5 (53, 6) (53, 3)
group4 (20, 6) (20, 3)
group3 (20, 6) (20, 3)
group2 (18, 6) (18, 3)
group1 (64, 6) (64, 3)
group0 (17, 6) (17, 3)


In [6]:
def create_dataset_tensor(dataset, filename):
    dataset_as_dict = {}
    embeddings_list = []
    for i, row in dataset.iterrows():
        fileobj = np.load(row["filepath"])
        embed_fileobj = np.load(row["embed_filepath"])
        embed_fileobj2 = np.load(row["embed_filepath2"])
        config_runtime = fileobj["config_runtime"]
        target = np.argsort(np.argsort(-config_runtime))
        embeddings1 = embed_fileobj["embeddings"]
        embeddings2 = embed_fileobj["embeddings"]
        embeddings = np.concatenate([embeddings1, embeddings2], axis=1)

        dataset_as_dict[i] = {
            "arch": row["arch"],
            "perm": row["perm"],
            "filename": row["filename"],
            "target": target,
            "X": embeddings,
        }
        embeddings_list.append(embeddings)
    embeddings_list = np.concatenate(embeddings_list, axis=0)

    emb_scl = embeddings_list.max(axis=0) - embeddings_list.min(axis=0)
    emb_mean = embeddings_list.mean(axis=0)
    del embeddings_list

    for i in dataset_as_dict:
        dataset_as_dict[i]["X"] = (dataset_as_dict[i]["X"] - emb_mean) / emb_scl

    with open(workdir / f"{filename}.json", "w") as f:
        json.dump({"xmean": emb_mean.tolist(), "xscl": emb_scl.tolist()}, f, indent=4)
    return dataset_as_dict


def create_valid_dataset_tensor(dataset, filename):
    dataset_as_dict = {}
    for i, row in tqdm(dataset.iterrows()):
        fileobj = np.load(row["filepath"])
        embed_fileobj = np.load(row["embed_filepath"])
        embed_fileobj2 = np.load(row["embed_filepath2"])
        config_runtime = fileobj["config_runtime"]
        target = np.argsort(np.argsort(-config_runtime))
        embeddings1 = embed_fileobj["embeddings"]
        embeddings2 = embed_fileobj2["embeddings"]
        embeddings = np.concatenate([embeddings1, embeddings2], axis=1)

        dataset_as_dict[i] = {
            "arch": row["arch"],
            "perm": row["perm"],
            "filename": row["filename"],
            "target": target,
            "X": embeddings,
        }

    with open(workdir / f"{filename}.json", "r") as f:
        scler = json.load(f)
        emb_scl, emb_mean = np.array(scler["xscl"]), np.array(scler["xmean"])

    for i in dataset_as_dict:
        dataset_as_dict[i]["X"] = (dataset_as_dict[i]["X"] - emb_mean) / emb_scl

    return dataset_as_dict

In [25]:
@dataclass
class Params:
    device: str
    dims: list[int] = field(default_factory=lambda: [512, 512])
    epoch: int = 500
    T_max: int = 500
    eta_min: float = 0
    lr: float = 1e-5
    weight_decay: float = 0
    grad_clip_max_norm: float = 1.0
    grad_clip_norm_type: float = 2.0
    dropout_p: float = 0.05

    sample_size: int = 1000
    batch_size: int = 8

    num_feats: int = 192 * 2


params = Params(device=GPU if torch.cuda.is_available() else "cpu")


class FineTuningDataset(Dataset):
    def __init__(
        self,
        dataset_as_dict: dict[str, Any],
        params: Params,
    ) -> None:
        self.dataset_as_dict = dataset_as_dict
        self.params = params

    @property
    def device(self) -> str:
        return self.params.device

    def __len__(self) -> int:
        return len(self.dataset_as_dict)

    def __getitem__(self, idx: int) -> tuple[torch.Tensor, torch.Tensor]:
        dataset = self.dataset_as_dict[idx]
        num_configs = dataset["target"].shape[0]
        indexes = random.choices(list(range(num_configs)), k=self.params.sample_size)

        embeddings = torch.tensor(
            dataset["X"][indexes, :],
            dtype=torch.float32,
        ).to(self.device)
        target = torch.tensor(
            dataset["target"][indexes],
            dtype=torch.float32,
        ).to(self.device)

        return embeddings, target

    def get_entire(self, idx: int) -> tuple[torch.Tensor, torch.Tensor]:
        dataset = self.dataset_as_dict[idx]
        embeddings = torch.tensor(
            dataset["X"],
            dtype=torch.float32,
        ).to(self.device)
        target = torch.tensor(
            dataset["target"],
            dtype=torch.float32,
        ).to(self.device)

        return embeddings, target

    def get_info(self, idx):
        dataset = self.dataset_as_dict[idx]
        return dataset["arch"], dataset["perm"], dataset["filename"]


class MLP(torch.nn.Module):
    def __init__(
        self,
        params: Params,
    ) -> None:
        super().__init__()
        self.params = params

        dims = [params.num_feats] + self.params.dims
        fc_layer = []
        for i in range(len(dims) - 1):
            fc_layer += [
                nn.Dropout(self.params.dropout_p),
                nn.Linear(
                    in_features=dims[i],
                    out_features=dims[i + 1],
                ),
                nn.ReLU(),
            ]
        fc_layer += [
            nn.Dropout(self.params.dropout_p),
            nn.Linear(
                in_features=dims[-1],
                out_features=1,
            ),
        ]

        self.net = nn.Sequential(*fc_layer)
        self.to(self.params.device)

    def forward(self, x) -> torch.Tensor:
        return self.net(x).squeeze()

In [13]:
def rankNet(y_pred, y_true):
    """
    RankNet loss introduced in "Learning to Rank using Gradient Descent".
    :param y_pred: predictions from the model, shape [batch_size, slate_length]
    :param y_true: ground truth labels, shape [batch_size, slate_length]
    :return: loss value, a torch.Tensor
    """
    y_pred = y_pred.clone()
    y_true = y_true.clone()

    document_pairs_candidates = list(itertools.combinations(range(y_true.shape[1]), 2))

    pairs_true = y_true[:, document_pairs_candidates]
    selected_pred = y_pred[:, document_pairs_candidates]

    true_diffs = pairs_true[:, :, 0] - pairs_true[:, :, 1]
    pred_diffs = selected_pred[:, :, 0] - selected_pred[:, :, 1]

    the_mask = (true_diffs > 0) & (~torch.isinf(true_diffs))
    pred_diffs = pred_diffs[the_mask]

    true_diffs = (true_diffs > 0).type(torch.float32)
    true_diffs = true_diffs[the_mask]

    return nn.BCEWithLogitsLoss()(pred_diffs, true_diffs)


class ListMLE(nn.Module):
    def __init__(self) -> None:
        super().__init__()

    def forward(self, logits: torch.Tensor, labels: torch.Tensor) -> torch.Tensor:
        """

        Parameters
        ----------
        logits: torch.Tensor
            予測（バッチサイズ, 要素数, ）
        labels: torch.Tensor
            目的変数（バッチサイズ, 要素数, ）

        Returns
        -------
        torch.Tensor
        """
        # 正解をソート
        labels_sorted, labels_sorted_indice = labels.sort(descending=True, dim=1)
        # 予測を正解順でソート
        logits_sorted_by_true = torch.gather(logits, dim=1, index=labels_sorted_indice)
        # 予測値の最大値で予測値を引く（expの爆発予防）
        logits_max, _ = logits_sorted_by_true.max(dim=1, keepdim=True)
        logits_sorted_by_true = logits_sorted_by_true - logits_max
        # ランキングが低いものから累積する(その後正解順に戻す)
        cumsums = torch.cumsum(logits_sorted_by_true.exp().flip(dims=[1]), dim=1).flip(
            dims=[1]
        )
        # 誤差
        negative_log_likelihood = torch.sum(
            torch.log(cumsums) - logits_sorted_by_true, dim=1
        )
        return torch.mean(negative_log_likelihood)

In [14]:
def seed_everything(seed=1234):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True


def to_cpu_numpy(
    params: Params, pred: torch.Tensor, truth: torch.Tensor
) -> tuple[np.ndarray, np.ndarray]:
    if params.device == GPU:
        pred_ = pred.cpu().detach().numpy()
        truth_ = truth.cpu().detach().numpy()
        torch.cuda.empty_cache()
    else:
        pred_ = pred.detach().numpy()
        truth_ = truth.detach().numpy()
    return pred_, truth_


def train(params, train_dataset_as_dict, valid_dataset_as_dict, savedir):
    train_dataset = FineTuningDataset(
        dataset_as_dict=train_dataset_as_dict, params=params
    )
    valid_dataset = FineTuningDataset(
        dataset_as_dict=valid_dataset_as_dict, params=params
    )
    train_dataloader = DataLoader(
        train_dataset, batch_size=params.batch_size, shuffle=True
    )

    model = MLP(params=params)
    optimizer = torch.optim.Adam(
        model.parameters(), lr=params.lr, weight_decay=params.weight_decay
    )
    scheduler = CosineAnnealingLR(
        optimizer=optimizer, T_max=params.T_max, eta_min=params.eta_min
    )
    criterion = ListMLE()

    pbar = tqdm(range(params.epoch))
    num_train_log, num_valid_log = 0, 0
    best_score = -np.inf
    best_epoch = None
    for epoch in range(params.epoch):
        is_log = (epoch + 1) % 10 == 0
        model.train()

        num_iters = len(train_dataloader)
        train_scores = []
        for i_iter, (X, target) in enumerate(train_dataloader):
            pred = model(X)

            if (len(pred.shape) == 1) or (len(target.shape) == 1):
                pred, target = pred.reshape(1, -1), target.reshape(1, -1)
            loss = criterion(pred, target)
            loss.backward()
            nn.utils.clip_grad_norm_(
                model.parameters(),
                max_norm=params.grad_clip_max_norm,
                norm_type=params.grad_clip_norm_type,
            )
            optimizer.step()
            scheduler.step(epoch + i_iter / num_iters)
            optimizer.zero_grad()

            pred, target = to_cpu_numpy(params, pred, target)

            scores = []
            for pred_, target_ in zip(pred, target):
                score = kendalltau(target_, pred_).correlation
                if is_log:
                    # wandb.log(
                    #     {
                    #         "epoch": epoch,
                    #         "iter": i_iter,
                    #         "lr": scheduler.get_last_lr()[0],
                    #         "train/score": score,
                    #         "train/loss": loss,
                    #         "train/pred": pred,
                    #         "train/target": target,
                    #         "train/count": num_train_log,
                    #     }
                    # )
                    num_train_log += 1
                scores.append(score)
            scores = np.array(scores)
            train_scores.append(scores)
        train_scores = np.hstack(train_scores)

        model.eval()
        losses, scores = [], []
        for i_graph in range(len(valid_dataset)):
            arch, perm, filename = valid_dataset.get_info(i_graph)
            X, target = valid_dataset[i_graph]
            pred = model(X)
            loss = criterion(pred.reshape(1, -1), target.reshape(1, -1))
            graph_loss = loss.item()
            pred, target = to_cpu_numpy(params, pred, target)
            score = kendalltau(target, pred).correlation

            if is_log:
                # wandb.log(
                #     {
                #         "epoch": epoch,
                #         "iter": i_iter,
                #         "lr": scheduler.get_last_lr()[0],
                #         "valid/arch": arch,
                #         "valid/perm": perm,
                #         "valid/filename": filename,
                #         "valid/score": score,
                #         "valid/loss": graph_loss,
                #         "valid/pred": pred,
                #         "valid/target": target,
                #         "valid/count": num_valid_log,
                #     }
                # )
                num_valid_log += 1
            losses.append(graph_loss)
            scores.append(score)
        losses = np.array(losses)
        scores = np.array(scores)
        score_ = np.mean(scores[~np.isnan(scores)])
        # print(
        #     f"[{epoch + 1}] valid score={np.mean(scores[~np.isnan(scores)]):.3f} valid loss={np.mean(losses[~np.isnan(losses)]):5f}"
        # )

        if best_score < score_:
            best_score = score_
            best_epoch = epoch
            torch.save(model.state_dict(), savedir / f"best_model.pt")

        if is_log:
            torch.save(model.state_dict(), savedir / f"epoch{epoch + 1}_model.pt")
        pbar.update(1)
        pbar.set_description(
            f"[{epoch + 1}] train score={np.mean(train_scores[~np.isnan(train_scores)]):.3f} valid score={score_:.3f} best = {best_score:.3f} [{best_epoch}] valid loss={np.mean(losses[~np.isnan(losses)]):1f}"
        )

In [21]:
group_name = "group15"
seed = 0
savedir = workdir / f"{group_name}" / f"seed{seed}"
savedir.mkdir(exist_ok=True, parents=True)

dataset = group_dict[group_name].copy()

valid_indexes = random.sample(list(range(dataset.shape[0])), dataset.shape[0] // 2)
is_valids = np.zeros(dataset.shape[0]).astype(int)
is_valids[valid_indexes] = 1
dataset["valid"] = is_valids
dataset.to_csv(savedir / "dataset.csv", index=False)

dftrain, dfvalid = dataset.query("valid == 0").reset_index(drop=True), dataset.query(
    "valid == 1"
).reset_index(drop=True)

train_dataset_as_dict = create_dataset_tensor(
    dataset=dftrain, filename=f"{group_name}-{seed}"
)
valid_dataset_as_dict = create_valid_dataset_tensor(
    dataset=dfvalid, filename=f"{group_name}-{seed}"
)

exptname = f"1113-listmle-{group_name}-seed{seed}"

# wandb.init(
#     project="predict-ai-model-runtime-for-sun-scan-clan",
#     config={
#         "params": asdict(params),
#     },
#     name=exptname,
# )

for i in range(10):
    model_savedir = savedir / f"model{i + 1}"
    model_savedir.mkdir(exist_ok=True, parents=True)
    seed_everything(i)
    train(
        params=params,
        train_dataset_as_dict=train_dataset_as_dict,
        valid_dataset_as_dict=valid_dataset_as_dict,
        savedir=model_savedir,
    )

# wandb.finish()

0it [00:00, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

/home/yamaguchi/.pyenv/versions/3.11.5/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:149: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

...

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

In [26]:
valid_dataset = FineTuningDataset(dataset_as_dict=valid_dataset_as_dict, params=params)

In [56]:
records = []
for i_data in range(len(valid_dataset)):
    X, target = valid_dataset.get_entire(i_data)
    target = target.cpu().detach().numpy()

    preds = []
    record = {}
    for i in range(10):
        model_savedir = savedir / f"model{i + 1}"
        model = MLP(params=params)
        model.load_state_dict(torch.load(model_savedir / f"best_model.pt"))
        model.eval()

        pred = model(X)
        pred = pred.cpu().detach().numpy()
        preds.append(pred)
        entile_score = kendalltau(target, pred).correlation

        sampled_score = []
        for _ in range(100):
            indexes = random.sample(list(range(pred.shape[0])), 1000)
            score_ = kendalltau(target[indexes], pred[indexes]).correlation
            sampled_score.append(score_)
        sampled_score = np.mean(sampled_score)
        record.update(
            {
                f"entire{i + 1}": entile_score,
                f"sampled{i + 1}": sampled_score,
            }
        )
    preds = np.vstack(preds)
    preds_scl = (preds - preds.min(axis=1, keepdims=True)) / (
        preds.max(axis=1, keepdims=True) - preds.min(axis=1, keepdims=True)
    )
    mean_ens_preds = preds_scl.mean(axis=0)
    med_ens_preds = np.median(preds_scl, axis=0)

    sampled_score_mean, sampled_score_med = [], []
    for _ in range(100):
        indexes = random.sample(list(range(pred.shape[0])), 1000)
        score_mean_ = kendalltau(target[indexes], mean_ens_preds[indexes]).correlation
        score_med_ = kendalltau(target[indexes], med_ens_preds[indexes]).correlation
        sampled_score_mean.append(score_mean_)
        sampled_score_med.append(score_med_)
    sampled_score_mean = np.mean(sampled_score_mean)
    sampled_score_med = np.mean(sampled_score_med)

    record.update(
        {
            f"entire_mean": kendalltau(target, mean_ens_preds).correlation,
            f"entire_median": kendalltau(target, med_ens_preds).correlation,
            f"sampled_mean": sampled_score_mean,
            f"sampled_median": sampled_score_med,
        }
    )
    records.append(record)
dfresult = pd.DataFrame(records)

In [57]:
dfresult

,entire1,sampled1,entire2,sampled2,entire3,sampled3,entire4,sampled4,entire5,sampled5,...,entire8,sampled8,entire9,sampled9,entire10,sampled10,entire_mean,entire_median,sampled_mean,sampled_median
0,0.341354,0.342989,0.331333,0.333615,0.317338,0.319576,0.354466,0.355693,0.332764,0.332677,...,0.345949,0.347477,0.352336,0.356040,0.422747,0.423232,0.354607,0.342401,0.356103,0.344025
1,0.524238,0.524876,0.523609,0.524165,0.522498,0.523299,0.525656,0.525964,0.523192,0.525280,...,0.525400,0.524948,0.525359,0.523599,0.530481,0.530456,0.525682,0.524240,0.525487,0.524017
2,0.556700,0.557642,0.556164,0.557075,0.554411,0.552543,0.558315,0.558633,0.555112,0.553745,...,0.558293,0.557991,0.559344,0.560466,0.570771,0.569383,0.558576,0.556759,0.558897,0.557082
3,0.227062,0.228606,0.220645,0.221563,0.201269,0.203408,0.226008,0.226568,0.211776,0.210387,...,0.234663,0.233910,0.241621,0.242529,0.242816,0.244233,0.236170,0.223772,0.239504,0.226853
4,0.117306,0.117956,0.116972,0.114313,0.112566,0.113661,0.124229,0.123438,0.114300,0.113554,...,0.124980,0.123742,0.125152,0.125769,0.142887,0.145897,0.124762,0.120125,0.125475,0.120913
5,0.131182,0.132426,0.130872,0.134556,0.127823,0.128243,0.138201,0.140684,0.128870,0.127882,...,0.137673,0.138123,0.138535,0.137059,0.154532,0.153192,0.138370,0.133729,0.140402,0.135769
6,0.167834,0.168709,0.167127,0.169579,0.163244,0.161303,0.174393,0.173594,0.165423,0.162420,...,0.177137,0.175541,0.177514,0.177009,0.196480,0.197900,0.176387,0.170041,0.176543,0.170196
7,0.202355,0.202509,0.199156,0.201235,0.181823,0.181454,0.207040,0.208180,0.187556,0.187764,...,0.198570,0.197626,0.196908,0.192841,0.211638,0.213956,0.209651,0.196571,0.210447,0.197447
8,0.216697,0.216497,0.216262,0.215565,0.218955,0.217388,0.218272,0.215152,0.215954,0.217160,...,0.218747,0.217120,0.216440,0.219369,0.222786,0.222013,0.218867,0.217590,0.219458,0.218163
9,0.513994,0.515226,0.514476,0.513908,0.499640,0.498421,0.526237,0.526517,0.499519,0.498867,...,0.524726,0.525678,0.526726,0.525706,0.591176,0.591185,0.525814,0.516360,0.527169,0.517838


In [61]:
dfresult.max(axis=1)

0     0.423232
1     0.530481
2     0.570771
3     0.244233
4     0.145897
5     0.154532
6     0.197900
7     0.213956
8     0.222786
9     0.591185
10    0.281616
11    0.432858
12    0.541023
13    0.416310
14    0.554306
15    0.307786
16    0.309936
17    0.350704
18    0.336575
19    0.147811
20    0.014055
21    0.580328
22    0.081743
23    0.171104
24    0.330634
dtype: float64

0.3546072749552976

In [35]:
sampled_score

0.33974317241987534

In [ ]:
for seed in range(43, 43 + 10):
    savedir = workdir / f"{group_name}" / f"seed{seed}"
    model_savedir = savedir / "model"

    dataset = pd.read_csv(savedir / "dataset.csv")
    # valid_indexes = random.sample(list(range(dataset.shape[0])), dataset.shape[0] // 2)
    # is_valids = np.zeros(dataset.shape[0]).astype(int)
    # is_valids[valid_indexes] = 1
    # dataset["valid"] = is_valids
    # dataset.to_csv(savedir / "dataset.csv", index=False)

    # dftrain, dfvalid = dataset.query("valid == 0").reset_index(
    #     drop=True
    # ), dataset.query("valid == 1").reset_index(drop=True)

    # train_dataset_as_dict = create_dataset_tensor(dataset=dftrain, filename=group_name)
    # valid_dataset_as_dict = create_valid_dataset_tensor(
    #     dataset=dfvalid, filename=group_name
    # )

    # train(
    #     params=params,
    #     train_dataset_as_dict=train_dataset_as_dict,
    #     valid_dataset_as_dict=valid_dataset_as_dict,
    #     savedir=model_savedir,
    # )

In [ ]:
# seed = 43
# for group_name in group_dict:
#     dataset = group_dict[group_name]
#     dataset_as_dict = create_dataset_tensor(dataset=dataset, filename=group_name)

#     exptname = f"1113-勾配クリップ-ミスマッチ-listmle-finetuning-{group_name}-seed{seed}"

#     wandb.init(
#         project="predict-ai-model-runtime-for-sun-scan-clan",
#         config={
#             "params": asdict(params),
#         },
#         name=exptname,
#     )

#     seed_everything(seed)

#     savedir = workdir / f"{group_name}" / f"seed{seed}"
#     savedir.mkdir(exist_ok=True, parents=True)
#     train(params=params, dataset_as_dict=dataset_as_dict, savedir=savedir)

#     wandb.finish()